In [3]:
import pandas as pd
import numpy as np
import datetime
from math import radians, cos, sin, asin, sqrt

def coord_distance(coord1, coord2):
    """
    CREDITS TO GEEKSFORGEEKS.ORG: https://www.geeksforgeeks.org/program-distance-two-points-earth/
        Function that calculates and returns the distance in meters between two coordinates
    Args:
        coord1 (numpy.float64, numpy.float64): tuple with latitude & longitude (in that order) for the first coordinate
        coord2 (numpy.float64, numpy.float64): tuple with latitude & longitude (in that order) for the second coordinate
    Returns:
        dist (float): distance in meters
    Example call:
        coord_distance((52.195248, 21.048823), (52.208840, 21.007883))
    """

    lon1 = radians(coord1[1])
    lon2 = radians(coord2[1])
    lat1 = radians(coord1[0])
    lat2 = radians(coord2[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2

    c = 2 * asin(sqrt(a))

    # Radius of earth in meters. Use 3956 for miles
    r = 6371000

    return (c * r)


In [4]:
from math import sqrt, atan2, pi
from random import randint
import matplotlib.pyplot as plt
import numpy as np

def make_line(coord1, S):
    """
    Returns the equation of the line that passes through coord1 and S
    :param coord1: (latitude, longitude) tuple -- is either A or B in the ASB method
    :param S: (latitude, longitude) tuple -- these are the coordinates of the busstop
    :return: (a, b, c) tuple -- parameters of the equation of the line passing through coord1 and S
                (where ax + by + c = 0)
    """
    x1, y1 = coord1
    x2, y2 = S
    a = y1 - y2
    b = x2 - x1
    c = (x1 - x2) * y1 + (y2 - y1) * x1
    return a, b, c

def lines_intersection(line1, line2):
    """
    Get the coordinate at which two lines intersect
    Used to assert that the lines cross at the busstop S
    :param line1: (a1, b1, c1) tuple -- parameters of the equation of line 1
    :param line2: (a2, b2, c2) tuple -- parameters of the equation of line 2
    :return: (latitude, longitude) tuple -- coordinate of the intersection of line 1 and 2
    """
    a1, b1, c1 = line1
    a2, b2, c2 = line2
    x = (b1 * c2 - b2 * c1) / (a1 * b2 - a2 * b1)
    y = (c1 * a2 - a1 * c2) / (a1 * b2 - a2 * b1)
    return x, y

def distance(coord1, coord2):
    x1, y1 = coord1
    x2, y2 = coord2
    return sqrt(((x1 - x2) ** 2) + ((y1 - y2) ** 2))


def get_angle_bisector(A, S, B):
    """
    Finds and returns the equation of the line that bisects the lines between A-S and S-B such that the angles
    between those lines are equal
    :param S: (latitude, longitude) tuple -- busstop coordinates
    :param A: (latitude, longitude) tuple -- first time in proximity of busstop coordinates
    :param B: (latitude, longitude) tuple -- last time in proximity of busstop coordinates
    :return: (a, b, c) tuple -- parameters of the equation of the angle bisector of lines A-S and S-B
    """

    # below is not completely correct, but an approximation to the angle bisector;
    # works if A and B do not have distances from S that are too different
    # Since in our case A and B are both approx r (=300?) away from S, this should work.
    midpoint_AB = ((A[0] + B[0]) / 2, (A[1] + B[1]) / 2)
    return make_line(midpoint_AB, S)

    # correct way, but does not work for all cases for some reason...
    """ 
    a1, b1, c1 = make_line(A, S)
    a2, b2, c2 = make_line(B, S)
    print(S)
    print(lines_intersection((a1, b1, c1), (a2, b2, c2)))
    denom1 = sqrt(a1 ** 2 + b1 ** 2)
    denom2 = sqrt(a2 ** 2 + b2 ** 2)
    # angle between A, S, and B
    angle_ASB = atan2(B[1] - S[1], B[0] - S[0]) - atan2(A[1] - S[1], A[0] - S[0])
    ASB_is_obtuse = False  # flag whether ASB angle > 0.5*pi
    if abs(angle_ASB) > 0.5 * pi:
        ASB_is_obtuse = True
    bisector1 = (a1 / denom1 - a2 / denom2,
                 b1 / denom1 - b2 / denom2,
                 c1 / denom1 - c2 / denom2)
    # if bisector1 has tan(theta) > 1, then it bisects the obtuse angle
    if abs((a1 * bisector1[1] - bisector1[0] * b1) / (a1 * bisector1[0] + b1 * bisector1[1])) > 1 and ASB_is_obtuse:
        return bisector1
    else:
        bisector2 = (a1 / denom1 + a2 / denom2,
                     b1 / denom1 + b2 / denom2,
                     c1 / denom1 + c2 / denom2)
        return bisector2
    """


def is_past_angle_bisector(A, coord, angle_bisector):
    """
    Checks whether a coordinate is on the other side of the angle bisector compared to A
    :param A: (latitude, longitude) tuple --  the coordinates where the bus is coming from (coordinates of when the bus
    enters the circle around the busstop for the first time)
    :param coord: (latitude, longitude) tuple -- the coordinates for which we want to check
    whether it is on the other side of the line compared to a or not
    :param angle_bisector: (a, b, c) tuple -- parameters of the equation of the angle bisector
    :return: True if coord is on the other side of the line compared to A, False otherwise
    """
    a1, b1, c1 = angle_bisector
    value_A = a1 * A[0] + b1 * A[1] + c1
    value_coord = a1 * coord[0] + b1 * coord[1] + c1

    if (value_A <= 0 and value_coord <= 0) or (value_A >= 0 and value_coord >= 0):
        return False
    else:
        return True

def visualize_ASB(A, S, B, coord, R=2):
    # plot lines
    a1, b1, c1 = make_line(A, S)
    a2, b2, c2 = make_line(B, S)

    # line AS
    x1 = np.linspace(S[0] - R * 1.25, S[0] + R * 1.25, 100)
    y1 = -a1/b1 * x1 - c1/b1
    plt.plot(x1, y1, '-g', label='line AS')

    # line BS
    x2 = np.linspace(S[0] - R * 1.25, S[0] + R * 1.25, 100)
    y2 = -a2/b2 * x2 - c2/b2
    plt.plot(x2, y2, '-b', label='line BS')

    # angle bisector
    a3, b3, c3 = get_angle_bisector(A, S, B)
    x3 = np.linspace(S[0] - R * 1.25, S[0] + R * 1.25, 100)
    y3 = -a3 / b3 * x3 - c3 / b3
    plt.plot(x3, y3, '-r', label='Angle Bisector')

    # A, S, B
    plt.plot(A[0], A[1], 'go')
    plt.plot(B[0], B[1], 'bo')
    plt.plot(S[0], S[1], 'ro')

    # coord
    plt.plot(coord[0], coord[1], 'ro')

    plt.title(f'Coord is past angle bisector? {is_past_angle_bisector(A, coord, (a3, b3, c3))}')

    # circle with radius R centered around S
    """ TODO
    theta = np.linspace(0, 2 * np.pi, 100)
    x1 = R * np.cos(theta)
    x2 = R * np.sin(theta)
    plt.plot(x1, x2, color='black')
    """

    plt.xlabel('x', color='#1C2833')
    plt.ylabel('y', color='#1C2833')
    plt.legend(loc='upper left')
    plt.grid()
    plt.show()

Import data, create new column with the line of each bus by extracting the first part of the trip_id field. 

In [5]:
import pandas as pd
data = pd.read_csv("stop_times.txt", sep=",", header=None)
data=data.rename(data.iloc[0],axis=1)
data=data[1:]
trip_id=data['trip_id']
x=data['trip_id'].str.split("_",expand=True)
lines=x[0]
data['Lines']=lines

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Create a dictionary which contains lines as keys and stops for each line as values

In [6]:
busstops_per_line={}
df=data
for i in range(len(df)):
    if df['Lines'].iloc[i] not in busstops_per_line.keys():
        busstops_per_line[df['Lines'].iloc[i]]=[df['stop_id'].iloc[i]]
    elif df['Lines'].iloc[i] in busstops_per_line.keys():
        if df['stop_id'].iloc[i] not in busstops_per_line[df['Lines'].iloc[i]]:
            busstops_per_line.setdefault(df['Lines'].iloc[i]).append(df['stop_id'].iloc[i])

#set up a python dictionary that holds the reported coordinates of _bus_ when _bus_ is within R meters of _S_:
asb_dict = defaultdict(list)

In [32]:
stops= pd.read_csv("stops.txt", sep=",", header=None)
stops=stops.rename(stops.iloc[0],axis=1)
stops=stops[1:]
import pandas as pd
locations=pd.read_csv("t_2020_09_01_22_30",sep=";", header=None)
locations['Longitude']=locations.iloc[:,3].rename('Longitude',axis=1)
locations['Latitude']=locations.iloc[:,4].rename('Latitude',axis=1)

locations.iloc[:,5]=locations.iloc[:,5].str.replace("T", " ")
locations.iloc[:,5]=locations.iloc[:,5].str[:-4]

Reduce the size of both datasets for faster trials

In [33]:
#locations=locations[0:500]
locations[0:3]


,0,1,2,3,4,5,Longitude,Latitude
0,196,2.0,2020-09-01 21:29:58,21.162977,52.260264,2020-09-01 21:30:12,21.162977,52.260264
1,311,1.0,2020-09-01 21:09:59,21.115079,52.234589,2020-09-01 21:30:18,21.115079,52.234589
2,222,5.0,2020-09-01 19:10:25,20.995193,52.186562,2020-09-01 21:30:18,20.995193,52.186562


Dictionary that contains the coordinates of all bus stops

In [34]:
print(stops)

      stop_id               stop_name   stop_lat   stop_lon  \
1     1001_01             Kijowska 01  52.248455  21.044827   
2     1001_02             Kijowska 02  52.249078  21.044443   
3     1001_03             Kijowska 03  52.248998  21.043983   
4     1001_04             Kijowska 04  52.249905  21.041726   
5     1001_06             Kijowska 06  52.250008  21.043710   
...       ...                     ...        ...        ...   
6645  R-11_99       ZEA KLESZCZOWA 99  52.196605  20.922746   
6646  R-13_00          ZEA STALOWA 00  52.263590  21.047868   
6647  R-13_99          ZEA STALOWA 99  52.263631  21.047922   
6648  R-19_00  WYDZIAŁ WŁOŚCIAŃSKA 00  52.271236  20.968586   
6649  R-19_99  WYDZIAŁ WŁOŚCIAŃSKA 99  52.271236  20.968496   

     wheelchair_boarding  
1                      1  
2                      1  
3                      1  
4                      1  
5                      1  
...                  ...  
6645                   0  
6646                   0  


In [35]:
stops_coordinates={}
for i in range(len(stops)):
    stops_coordinates[stops['stop_id'].iloc[i]]=(float(stops['stop_lat'].iloc[i]),float(stops['stop_lon'].iloc[i]))

In [36]:
print(len(stops_coordinates))
stops_coordinates['4033_02']

6649


(52.198991, 20.984341)

In [61]:
from datetime import date, datetime, timedelta
from collections import defaultdict
from time import sleep

asb_real = defaultdict(list)
arrival_time_estimations = []

for i in range(len(locations)):
    #print(arrival_time_estimations)
    
    bus_line=locations.iloc[i,0]
    bus_brigade=locations.iloc[i,1]
    
    
    if bus_line in busstops_per_line:
        stops = busstops_per_line[bus_line]
    # else: debug -- should rarely happen
        
    
    for stop in stops:
        if ((bus_line,bus_brigade),(stop)) in asb_real.keys(): # Keys are now ((bus,bus_brigade),(stop)) to make it clearer
            stop_coordinate=stops_coordinates[stop[1:]] # coordinates of the stop
            bus_coordinate=(float(locations['Latitude'].iloc[i]),float(locations['Longitude'].iloc[i]))
            distance=coord_distance(bus_coordinate,stop_coordinate)
            if distance < 300:
                
                # APPEND
                asb_real[((bus_line,bus_brigade),(stop))].append({'time':[locations.iloc[i,5]],'coord':[(bus_coordinate)]})
    
            else: # complete ASB

                #print(asb_real[((bus,bus_brigade),(stop))][0])  
                
                A=asb_real[((bus_line,bus_brigade),(stop))][0]['coord']
                
                if len(asb_real[((bus_line,bus_brigade),(stop))]) == 1:
                    arrival_time = asb_real[((bus_line,bus_brigade),(stop))][0]['time'][0]
                    #print(f'JUST ONE POINT -- TIME = {arrival_time}')
                    arrival_time_estimations.append([bus_line, bus_brigade,stop,arrival_time,None])
                    del asb_real[((bus_line,bus_brigade),(stop))]
                    continue
                
                elif len(asb_real[((bus_line,bus_brigade),(stop))]) == 2: # take average
                    
                    ts1 = asb_real[((bus_line,bus_brigade),(stop))][0]['time'][0]
                    ts2 = asb_real[((bus_line,bus_brigade),(stop))][1]['time'][0]
                    ts2_date = datetime.strptime(ts2, '%Y-%m-%d %H:%M:%S')
                    average_delta = (ts2_date - ts1_date) / 2
                    arrival_time = ts1_date + average_delta
                    #print(arrival_time)
                    arrival_time_estimations.append([bus_line, bus_brigade,stop,arrival_time,None])
                    del asb_real[((bus_line,bus_brigade),(stop))]
                    continue

                else:
                    #print(f' MULTIPLE STAMPS: {asb_real}')

                    #print(asb_real[((bus,bus_brigade),(stop))]['coord'])
                    
                    B=asb_real[((bus_line,bus_brigade),(stop))][-2]['coord']
                    
                
                    S = stop_coordinate
                    angle_bisector=get_angle_bisector(A[0], S, B[0])
                    index_past_bisector=0
                    
                    some_coord_past_bisector = False

                    for i in range(len(asb_real[((bus_line,bus_brigade),(stop))])):
                        
                        coord=asb_real[((bus_line,bus_brigade),(stop))][i]['coord']
                        #print(coord)

                        if is_past_angle_bisector(A[0],coord[0],angle_bisector):
                            #print(i)
                            index_past_bisector = i
                            index_before_bisector = i-1
                            some_coord_past_bisector = True
                            
                            # break out of for loop
                            break
                    

                    if not some_coord_past_bisector: # can happen when there are duplicate items in the list or bus is sitting at exact same location
                    # just take the time of entering the radius (might be somewhat optimistic but not a big problem)
                        arrival_time = asb_real[((bus_line,bus_brigade),(stop))][0]['time'][0]
                        #print(f' NO COORDS PAST BISECTOR -- {arrival_time}')
                        arrival_time_estimations.append([bus_line, bus_brigade,stop,arrival_time,None])
                        del asb_real[((bus_line,bus_brigade),(stop))]
                        continue
                    

                    time_before_S = asb_real[((bus_line,bus_brigade),(stop))][index_before_bisector]['time'][0]
                    time_after_S =  asb_real[((bus_line,bus_brigade),(stop))][index_past_bisector]['time'][0]
                    #print('TIME BEFORE AND AFTER BUSSTOP')
                    #print(time_before_S)
                    #print(time_after_S)
                    ts1_date = datetime.strptime(time_before_S, '%Y-%m-%d %H:%M:%S')
                    ts2_date = datetime.strptime(time_after_S, '%Y-%m-%d %H:%M:%S')
                    average_delta = (ts2_date - ts1_date) / 2
                    arrival_time = ts1_date + average_delta
                    #print(arrival_time)
                    arrival_time_estimations.append([bus_line, bus_brigade,stop,arrival_time,None])
                    del asb_real[((bus_line,bus_brigade),(stop))]
                    continue

        else:
            stop_coordinate=stops_coordinates[stop[1:]]
            bus_coordinate=(float(locations['Latitude'].iloc[i]),float(locations['Longitude'].iloc[i]))
            distance=coord_distance(bus_coordinate,stop_coordinate)

            if distance < 300:
                
                # APPEND
                asb_real[((bus_line,bus_brigade),(stop))].append({'time':[locations.iloc[i,5]],'coord':[(bus_coordinate)]})

In [46]:
['2020-09-01 21:30:37'][0]

'2020-09-01 21:30:37'

In [62]:
print(arrival_time_estimations[0:3])

[['739', 3.0, ' 3017_01', '2020-09-01 21:30:37', None], ['739', 3.0, ' 3018_01', '2020-09-01 21:30:37', None], ['739', 3.0, ' 3018_02', '2020-09-01 21:30:37', None]]


In [63]:
import csv
with open('arrival_estimations.csv', 'w') as arrival_times:
    arrival_writer = csv.writer(arrival_times, delimiter=',', quotechar='"')
    for i in range(len(arrival_time_estimations)):
        arrival_writer.writerow(arrival_time_estimations[i])